# Creating images using shapes and simple simulation with attenuation
This exercise shows how to create images via geometric shapes. It then uses forward projection without
and with attenuation.

It is recommended you complete the [Introductory](../Introductory) notebooks first (or alternatively the [display_and_projection.ipynb](display_and_projection.ipynb)). There is some overlap with [acquisition_model_mr_pet_ct.ipynb](../Introductory/acquisition_model_mr_pet_ct.ipynb), but here we use some geometric shapes to create an image and add attenuation etc.

Authors: Kris Thielemans and Evgueni Ovtchinnikov  
First version: 8th of September 2016  
Second Version: 17th of May 2018

CCP SyneRBI Synergistic Image Reconstruction Framework (SIRF).  
Copyright 2015 - 2017 Rutherford Appleton Laboratory STFC.  
Copyright 2015 - 2018 University College London.

This is software developed for the Collaborative Computational
Project in Synergistic Reconstruction for Biomedical Imaging
(http://www.ccpsynerbi.ac.uk/).

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
    http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# Initial set-up

In [ ]:
#%% make sure figures appears inline and animations works
%matplotlib notebook

# Setup the working directory for the notebook
import notebook_setup
from sirf_exercises import cd_to_working_dir
cd_to_working_dir('PET', 'image_creation_and_simulation')

In [ ]:
import notebook_setup

#%% Initial imports etc
import numpy
from numpy.linalg import norm
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import os
import sys
import shutil

In [ ]:
#%% Use the 'pet' prefix for all SIRF functions
# This is done here to explicitly differentiate between SIRF pet functions and 
# anything else.
import sirf.STIR as pet
from sirf.Utilities import show_2D_array, show_3D_array, examples_data_path
from sirf_exercises import exercises_data_path

In [ ]:
# define the directory with input files
data_path = os.path.join(examples_data_path('PET'), 'brain')

# Creation of images

In [ ]:
#%% Read in image
# We will use an image provided with the demo to have correct voxel-sizes etc
image = pet.ImageData(os.path.join(data_path, 'emission.hv'))
print(image.dimensions())
print(image.voxel_sizes())

In [ ]:
#%% create a shape
shape = pet.EllipticCylinder()
# define its size (in mm)
shape.set_length(50)
shape.set_radii((40, 30))
# centre of shape in (x,y,z) coordinates where (0,0,0) is centre of first plane
shape.set_origin((20, -30, 60))

In [ ]:
#%% add the shape to the image
# first set the image values to 0
image.fill(0)
image.add_shape(shape, scale=1)

In [ ]:
#%% add same shape at different location and with different intensity
shape.set_origin((40, -30, -60))
image.add_shape(shape, scale=0.75)

In [ ]:
#%% show the phantom image as a sequence of transverse images
show_3D_array(image.as_array());

# Simple simulation
Let's first do simple ray-tracing without attenuation

In [ ]:
#%% Create a SIRF acquisition model
acq_model = pet.AcquisitionModelUsingRayTracingMatrix()
# Specify sinogram dimensions via the template
template = pet.AcquisitionData(os.path.join(data_path, 'template_sinogram.hs'))
# Now set-up our acquisition model with all information that it needs about the data and image.
acq_model.set_up(template,image); 

In [ ]:
#%% forward project this image and display all sinograms
acquired_data_no_attn = acq_model.forward(image)
acquired_data_no_attn_array = acquired_data_no_attn.as_array()[0,:,:,:]
show_3D_array(acquired_data_no_attn_array);

In [ ]:
#%% Show every 8th view 
# Doing this here with a complicated one-liner...
show_3D_array(
    acquired_data_no_attn_array[:,0:acquired_data_no_attn_array.shape[1]:8,:].transpose(1,0,2),
    show=False)
# You could now of course try the animation of the previous demo...

# Adding attenuation
Attenuation in PET follows the Lambert-Beer law:

$$\exp\left\{-\int\mu(x) dx\right\},$$

with $\mu(x)$ the linear attenuation coefficients (roughly proportional to density), 
and the line integral being performed between the 2 detectors.

In SIRF, we model this via an `AcquisitionSensitivityModel` object. The rationale for the name is that attenuation reduces the sensitivity of the detector-pair.

In [ ]:
#%% create an attenuation image
# we will use the "emission" image as a template for sizes (although xy size doesn't have to be identical)
attn_image = image.get_uniform_copy(0)
#%% create a shape for a uniform cylinder in the centre
shape = pet.EllipticCylinder()
shape.set_length(150)
shape.set_radii((60, 60))
shape.set_origin((0, 0, 40))
# add it to the attenuation image with mu=-.096 cm^-1 (i.e. water)
attn_image.add_shape(shape, scale=0.096)

In [ ]:
#%% show the phantom image as a sequence of transverse images
show_3D_array(attn_image.as_array());

In [ ]:
#%% Create the acquisition sensitivity model
# First create the ray-tracer
acq_model_for_attn = pet.AcquisitionModelUsingRayTracingMatrix()
# Now create the attenuation model
asm_attn = pet.AcquisitionSensitivityModel(attn_image, acq_model_for_attn)

In [ ]:
attn_image.as_array().max()

In [ ]:
# Use this to find the 'detection efficiencies' as sinograms
asm_attn.set_up(template)
attn_factors = asm_attn.forward(template.get_uniform_copy(1))
# We will store these directly as an `AcquisitionSensitivityModel`, 
# such that we don't have to redo the line integrals
asm_attn = pet.AcquisitionSensitivityModel(attn_factors)

In [ ]:
#%% check a single sinogram (they are all the same of course)
show_2D_array('Attenuation factor sinogram', attn_factors.as_array()[0,5,:,:]);

In [ ]:
#%% check a profile (they are also all the same as the object is a cylinder in the centre)
plt.figure()
plt.plot(attn_factors.as_array()[0,5,0,:]);

In [ ]:
#%% Create a SIRF acquisition model
# start with ray-tracing
acq_model_with_attn = pet.AcquisitionModelUsingRayTracingMatrix()
# add the 'sensitivity'
acq_model_with_attn.set_acquisition_sensitivity(asm_attn)
# set-up
acq_model_with_attn.set_up(template,attn_image);

In [ ]:
#%% forward project the original image, now including attenuation modelling, and display all sinograms
acquired_data_with_attn = acq_model_with_attn.forward(image)
acquired_data_with_attn_array = acquired_data_with_attn.as_array()[0,:,:,:]
show_3D_array(acquired_data_with_attn_array);

In [ ]:
#%% Plot some profiles
slice = 40
plt.figure()
profile_no_attn = acquired_data_no_attn_array[5,slice,:]
profile_with_attn = acquired_data_with_attn_array[5,slice,:]
profile_attn_factors = attn_factors.as_array()[0,5,slice,:]

plt.plot(profile_no_attn,label='no atten')
plt.plot(profile_with_attn,label='with atten')
plt.plot(profile_no_attn * profile_attn_factors,'bo',label='check')
plt.legend();

# Further things to try
- Back project the data with and without attenuation
- Add noise to the data before backprojection  (not so easy unfortunately. Adding noise is done in the [ML_reconstruction](ML_reconstruction.ipynb) exercise).
Hint: use `acquired_data.clone()` to create a copy, `numpy.random.poisson`, and `acquired_data.fill()`.
- Add an additive background to the model. Check if it modifies the forward projection (it should!) and the back-projection?    
Hint: read the help for `AcquisitionModel`. Create a simple background by using `AcquisitionData.get_uniform_copy`.

In [ ]:
help(pet.AcquisitionModel)

In [ ]:
help(pet.AcquisitionData.get_uniform_copy)